Time to dig into the Blitz API - this is the more powerful API (afaik the one thing it doesn't do well is thumbnail/downsampled image generation). It uses Ice (a remote procedure framework) to talk directly to the OMERO server, rather than going through the web interface. The beginning is familiar: import configs, the `BlitzGateway` class, matplotlib, and here we are importing the whole `ezomero` library for all the helper functions we have already written. (if we didn't have `ezomero`, the list of things to be imported from `omero` would be MUCH longer).

In [ ]:
from config import OMEROUSER, OMEROPASS, OMEROHOST, OMEROPORT, OMEROWEB
import matplotlib.pyplot as plt
import ezomero

Again, pretty familiar: create `BlitzGateway` instance and connect it using the `ezomero.connect` helper function, print return object to make sure it worked.

In [ ]:
conn = ezomero.connect(OMEROUSER, OMEROPASS, port=OMEROPORT, host=OMEROHOST, group="", secure=True)
print(conn)

This whole block will have a lot of stuff about users and groups. We get the current user, check its ID, name and full name, check if we're an admin. Then, we list the groups we're a member of and the current group we are accessing. If you need data from a different group, you WILL need to change groups (we have a `set_group` helper function in `ezomero`). Then, we have a look at the current group's owners and which groups our current user owns.

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

# Check if you are an Administrator
print("   Is Admin:", conn.isAdmin())
if not conn.isFullAdmin():
    # If 'Restricted Administrator' show privileges
    print(conn.getCurrentAdminPrivileges())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())

# List the group owners and other members
owners, members = group.groupSummary()
print("   Group owners:")
for o in owners:
    print("     ID: %s %s Name: %s" % (
        o.getId(), o.getOmeName(), o.getFullName()))

print("Owner of:")
for g in conn.listOwnedGroups():
    print("   ID: ", g.getName(), " Name:", g.getId())


Let's start with where we ended in the previous section - retrieving the pixels of an image and plotting them. Here, we are plotting only the first timepoint/channel/slice of a 5d image. 

In [ ]:
image_id = 41252
img, pix = ezomero.get_image(conn, image_id, xyzct=True)
print(pix.shape)
plt.imshow(pix[:,:,0,0,0].T)

But how do you get an image id programatically? Glad you asked. (this is a good way to loop over all images in a dataset, for example.)

In [ ]:
dataset_id = 633
id_list = ezomero.get_image_ids(conn, dataset=dataset_id)
print(id_list)

You can also loop over all map annotations (i.e. key-value pairs)  in an image. 

In [ ]:
image_id = 41252
ann_ids = ezomero.get_map_annotation_ids(conn, 'Image', image_id)
print(ann_ids)

And then it's easy to print individual map annotations when you have their id. (if your image doesn't have any map annotations, you can create some via OMERO.web at ctomeroweb01.jax.org)

In [ ]:
ezomero.print_map_annotation(conn, ann_ids[0])

You can retrieve group ID and user ID by name using these helpful functions.

In [ ]:
group_id = ezomero.get_group_id(conn,'training')
print(group_id)

In [ ]:
user_id = ezomero.get_user_id(conn, 'user-1')
print(user_id)

You can also list all projects your current user has access to in their current group, and list all datasets in a given project as long as you have its id.

In [ ]:
ezomero.print_projects(conn)
proj_id = 441

In [ ]:
ezomero.print_datasets(conn, proj_id)

You can also use this API to easily create new stuff - in this case, we create a new dataset named `test_dataset` inside the project with given ID. 

In [ ]:
data_id = ezomero.post_dataset(conn, 'test_dataset', proj_id)

Finally, we will create a new image (from a random `numpy` array) and add it to our new dataset - furthermore, we will create a new map annotation and add it to the new image (we get both the dataset id and the image id as return values of the relevant functions)

In [ ]:
from numpy.random import rand
img = rand(1000,1000,1,1,1)
img_id = ezomero.post_image(conn, img, 'randomimage.tif', dataset_id=data_id)

In [ ]:
ann_id = ezomero.post_map_annotation(conn, 'Image', img_id, {"test": 1, "test2": 2}, 'test_ns')

We can also create ROIs using ezomero. That is slightly more involved: you need to create a list of `Shape`s that will be passed to the ROI, and these `Shape`s can be of many different types:

In [ ]:
from ezomero.rois import Point, Rectangle
shapes = []
point = Point(x=30.6, y=80.4)
shapes.append(point)
rectangle = Rectangle(x=50.0,
                          y=51.3,
                          width=90,
                          height=40,
                          z=0,
                          label='The place',
                          fill_color=(255, 10, 10, 150),
                          stroke_color=(255, 0, 0, 0),
                          stroke_width=2)
shapes.append(rectangle)
ezomero.post_roi(conn, img_id, shapes, name="My ROI")